In [9]:
# General required python packages
import os
import sys
from pathlib import Path
import numpy as np
import xarray as xr
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import cartopy as cpy

# Hackathon packages
import math as maths
import cartopy.crs as ccrs
import intake
import easygems.healpix as egh
import warnings

# Suppress specific FutureWarnings matching the message pattern when using cat[...].to_dask()
warnings.filterwarnings(
    "ignore",
    message=".*The return type of `Dataset.dims` will be changed.*",
    category=FutureWarning,
)

# Import the LoSSETT packages
from lossett_control.preprocessing import preprocess_era5
from lossett.calc.calc_inter_scale_transfers import calc_inter_scale_energy_transfer_kinetic

In [29]:
year = 2020
month = 8
day = 1
tsteps_per_day = 8
sampling = f"{int(24/tsteps_per_day)}h"
print(sampling)
lon_bound_field = "periodic"
lat_bound_field = np.nan

3h


In [11]:
# output directory
OUT_DIR = f"/gws/nopw/j04/kscale/USERS/dship/LoSSETT_out/ERA5"
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

In [12]:
 # calculation specification
max_r_deg = 3.0 # Maximum l to go up to in degrees
tsteps = 8
tchunks = 8
pchunks = 12
prec = 1e-10

In [13]:
control_dict = {
        "max_r": max_r_deg,
        "max_r_units": "deg",
        "angle_precision": prec,
        "x_coord_name": "longitude",
        "x_coord_units": "deg",
        "x_coord_boundary": lon_bound_field,
        "y_coord_name": "latitude",
        "y_coord_units": "deg",
        "y_coord_boundary": lat_bound_field
    }

Opening data:

In [30]:
cat = intake.open_catalog('https://digital-earths-global-hackathon.github.io/catalog/catalog.yaml')['online']
sim = cat['ERA5']
ds = sim(zoom=7).to_dask()
ds

<xarray.Dataset> Size: 545GB
Dimensions:  (time: 1008, cell: 196608, level: 37)
Coordinates:
  * cell     (cell) float32 786kB 0.0 1.0 2.0 ... 1.966e+05 1.966e+05 1.966e+05
    crs      float32 4B ...
    lat      (cell) float32 786kB dask.array<chunksize=(196608,), meta=np.ndarray>
  * level    (level) float32 148B 1.0 2.0 3.0 5.0 ... 925.0 950.0 975.0 1e+03
    lon      (cell) float32 786kB dask.array<chunksize=(196608,), meta=np.ndarray>
  * time     (time) datetime64[ns] 8kB 1940-01-01 1940-02-01 ... 2023-12-01
Data variables: (12/111)
    100u     (time, cell) float32 793MB dask.array<chunksize=(24, 16384), meta=np.ndarray>
    100v     (time, cell) float32 793MB dask.array<chunksize=(24, 16384), meta=np.ndarray>
    10fg     (time, cell) float32 793MB dask.array<chunksize=(24, 16384), meta=np.ndarray>
    10si     (time, cell) float32 793MB dask.array<chunksize=(24, 16384), meta=np.ndarray>
    10u      (time, cell) float32 793MB dask.array<chunksize=(24, 16384), meta=np.ndarray>
    10v      (time, cell) float32 793MB dask.array<chunksize=(24, 16384), meta=np.ndarray>
    ...       ...
    uvb      (time, cell) float32 793MB dask.array<chunksize=(24, 16384), meta=np.ndarray>
    v        (time, level, cell) float32 29GB dask.array<chunksize=(24, 4, 4096), meta=np.ndarray>
    vo       (time, level, cell) float32 29GB dask.array<chunksize=(24, 4, 4096), meta=np.ndarray>
    w        (time, level, cell) float32 29GB dask.array<chunksize=(24, 4, 4096), meta=np.ndarray>
    z        (time, level, cell) float32 29GB dask.array<chunksize=(24, 4, 4096), meta=np.ndarray>
    zust     (time, cell) float32 793MB dask.array<chunksize=(24, 16384), meta=np.ndarray>
Attributes:
    acknowledgment:  Contains modified Copernicus Climate Change Service info...
    contact:         lukas.kluft@mpimet.mpg.de
    creator:         Lukas Kluft
    description:     HEALPixelation of ERA5
    source:          Post-processed dataset based on the ERA5 mirror located ...

In [26]:
var_names, yearmonths, data_dir = preprocess_era5.setup_vars_yearmonth(
        year, month, sampling="3h", return_dates=False
    )

In [27]:
ds_u_3D = preprocess_era5.load_era5(var_names, yearmonths, data_dir)

In [28]:
ds_u_3D

<xarray.Dataset> Size: 12GB
Dimensions:    (time: 248, latitude: 360, pressure: 12, longitude: 720)
Coordinates:
  * time       (time) datetime64[ns] 2kB 2016-08-01T01:00:00 ... 2016-08-31T2...
  * latitude   (latitude) float64 3kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * pressure   (pressure) float64 96B 1e+03 925.0 850.0 ... 200.0 150.0 100.0
  * longitude  (longitude) float64 6kB -180.0 -179.5 -179.0 ... 179.0 179.5
Data variables:
    u          (time, pressure, latitude, longitude) float32 3GB dask.array<chunksize=(8, 12, 360, 720), meta=np.ndarray>
    v          (time, pressure, latitude, longitude) float32 3GB dask.array<chunksize=(8, 12, 360, 720), meta=np.ndarray>
    w          (time, pressure, latitude, longitude) float64 6GB dask.array<chunksize=(8, 12, 360, 720), meta=np.ndarray>
Attributes:
    CDI:                     Climate Data Interface version 2.4.0 (https://mp...
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    history:                 Thu Oct 10 15:18:29 2024: cdo remapcon,r720x360 ...
    CDO:                     Climate Data Operators version 2.4.0 (https://mp...